In [ ]:
import pandas as pd
from datetime import datetime, date, timedelta
import requests
from pytz import timezone

In [ ]:
import sqlalchemy
!pip install pymysql
import pymysql

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
PASSWORD = os.environ.get('PASSWORD1')
API_key = os.environ.get('API_key')

# Function for updating flight arrivals

In [ ]:
def flight_arrivals(icao_list):

    ## preparing strings of time data for input in URL
    ##  the .strftime function allows us to choose the format
    now = datetime.now().astimezone(timezone('Europe/Berlin')).now()
    ## as the now variable is the only of type datetime
    ## it is the only one used in combination with the .strftime method
    
    ## format time with .strftime() to match format for API call
    start_t = now.strftime("%Y-%m-%dT%H:%M")
    end_t = (now + timedelta(hours=12)).strftime("%Y-%m-%dT%H:%M")
    start_t2 = end_t
    end_t2 = (now + timedelta(days=1)).strftime("%Y-%m-%dT%H:%M")
    # today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
    # tomorrow = (today + timedelta(days=1))

    list_for_df = []
    
    for icao in icao_list:   # for loop for every airport of the list
        for i in range(2):   # for loop to consider each time period
            if i == 0:       # if clause to choose the time peruid
                url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{start_t}/{end_t}"
            # else:
                url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{start_t2}/{end_t2}"
            
            querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}          
            headers = {
            'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
            'x-rapidapi-key': API_key # <----- your API_key
            }
            response = requests.request("GET", url, headers=headers, params=querystring)
            flights_json = response.json()

                
            for flight in flights_json['arrivals']:
                flights_dict = {}
                
                # if flight['isCargo'] == False:
                
                # flights_dict['is cargo'] = []
                
                flights_dict['icao'] = icao
                
                # .get() is another way of ensuring our code doesn't break
                # in the previous 2 notebooks you learnt about 'if' (cities) and 'try/except' (weather)
                # .get() works similar, it will get the text if possible, if there is no text a None value will be inserted instead
                flights_dict['arrival_time_local'] = flight['arrival'].get('scheduledTimeLocal', None)
                flights_dict['arrival_time_Utc'] = flight['arrival'].get('scheduledTimeUtc', None)
                flights_dict['data_retrieved_on'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()
                              
                # creating a list of dictionaries that all have the same keys
                # therefore the dataframe will include the same informations under the same category
                list_for_df.append(flights_dict)

    return pd.DataFrame(list_for_df)

## Creating the dataframe for pushing in two steps

In [ ]:
icaos = ['EDDB']
arr_df = flight_arrivals(icaos).head()

### Adding the foreign key

In [ ]:
## inserting the empty column at the right place
airport_arrivals_df = arr_df.copy()
airport_arrivals_df.insert(1, "weather_FK", "", allow_duplicates=True)
airport_arrivals_df

## filling the column with values
for time in airport_arrivals_df.loc[:,"arrival_time_local"]:
    w_t = datetime.strptime(time,"%Y-%m-%d %H:%M+01:00")
    # weather interval
    weath_int = (int(w_t.strftime("%H")) // 3)*3
    suffix = w_t.strftime("%Y-%m-%d ")
    # print(f"{suffix}{weath_int}:00:00")
    airport_arrivals_df["weather_FK"] = f"{suffix}{weath_int}:00:00"

In [ ]:
airport_arrivals_df

,icao,weather_FK,arrival_time_local,arrival_time_Utc,data_retrieved_on
0,EDDB,2022-12-01 6:00:00,2022-12-01 07:55+01:00,2022-12-01 06:55Z,2022-11-30
1,EDDB,2022-12-01 6:00:00,2022-12-01 07:30+01:00,2022-12-01 06:30Z,2022-11-30
2,EDDB,2022-12-01 6:00:00,2022-12-01 07:50+01:00,2022-12-01 06:50Z,2022-11-30
3,EDDB,2022-12-01 6:00:00,2022-12-01 07:55+01:00,2022-12-01 06:55Z,2022-11-30
4,EDDB,2022-12-01 6:00:00,2022-12-01 07:55+01:00,2022-12-01 06:55Z,2022-11-30


### Cleaning all datetime values

In [ ]:
airport_arrivals_df["arrival_time_local"] = pd.to_datetime(airport_arrivals_df["arrival_time_local"])

In [ ]:
airport_arrivals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype                               
---  ------              --------------  -----                               
 0   icao                5 non-null      object                              
 1   weather_FK          5 non-null      object                              
 2   arrival_time_local  5 non-null      datetime64[ns, pytz.FixedOffset(60)]
 3   arrival_time_Utc    5 non-null      object                              
 4   data_retrieved_on   5 non-null      object                              
dtypes: datetime64[ns, pytz.FixedOffset(60)](1), object(4)
memory usage: 328.0+ bytes


In [ ]:
airport_arrivals_df["arrival_time_Utc"] = pd.to_datetime(airport_arrivals_df["arrival_time_Utc"])

In [ ]:
airport_arrivals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype                               
---  ------              --------------  -----                               
 0   icao                5 non-null      object                              
 1   weather_FK          5 non-null      object                              
 2   arrival_time_local  5 non-null      datetime64[ns, pytz.FixedOffset(60)]
 3   arrival_time_Utc    5 non-null      datetime64[ns, UTC]                 
 4   data_retrieved_on   5 non-null      object                              
dtypes: datetime64[ns, UTC](1), datetime64[ns, pytz.FixedOffset(60)](1), object(3)
memory usage: 328.0+ bytes


In [ ]:
airport_arrivals_df.dtypes

icao                                                object
weather_FK                                          object
arrival_time_local    datetime64[ns, pytz.FixedOffset(60)]
arrival_time_Utc                       datetime64[ns, UTC]
data_retrieved_on                                   object
dtype: object

In [ ]:
type(airport_arrivals_df["arrival_time_Utc"][0])

pandas._libs.tslibs.timestamps.Timestamp

In [ ]:
# working on giving all dates the format "%Y-%m-%d %H:%M
# I didn't need it in the end 

# for time, utc in zip(airport_arrivals_df.loc[:,["arrival_time_local"]],airport_arrivals_df.loc[:,["arrival_time_Utc"]]):
    
#     ## the following two lines give me an error that makes no sense
#     ##  time data 'arrival_time_local' does not match format '%Y-%m-%d %H:%M+01:00'
#     # it makes nos ense, because above it matched and now it doesnÄt match
#     l_t = datetime.strptime(time,"%Y-%m-%d %H:%M+01:00")
#     u_t_c = datetime.strptime(utc,"%Y-%m-%d %H:%M+01:00")
    
#     # to string
#     local = l_t.strftime("%Y-%m-%d %H:%M") #:%S")
#     Utc = u_t_c.strftime("%Y-%m-%d %H:%M") #:%S")
#     # print(f"{suffix}{weath_int}:00:00")
#     airport_arrivals_df["arrival_time_local"] = f"{local}"
#     airport_arrivals_df["arrival_time_Utc"] = f"{Utc}"

# Function for updating flight departures

In [ ]:
def flight_departures(icao_list):

    ## preparing strings of time data for input in URL
    ##  the .strftime function allows us to choose the format
    now = datetime.now().astimezone(timezone('Europe/Berlin')).now()
    ## as the now variable is the only of type datetime
    ## it is the only one used in combination with the .strftime method 
    start_t = now.strftime("%Y-%m-%dT%H:%M")
    end_t = (now + timedelta(hours=12)).strftime("%Y-%m-%dT%H:%M")
    start_t2 = end_t
    end_t2 = (now + timedelta(days=1)).strftime("%Y-%m-%dT%H:%M")

    list_for_df = []
    
    for icao in icao_list:   # for loop for every airport of the list
        for i in range(2):   # for loop to consider each time period
            if i == 0:       # if clause to choose the time peruid
                url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{start_t}/{end_t}"
            # else:
                # url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{start_t2}/{end_t2}"
            
            querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}          
            headers = {
            'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
            'x-rapidapi-key': API_key # <----- your API_key
            }
            response = requests.request("GET", url, headers=headers, params=querystring)
            flights_json = response.json()

                
            for flight in flights_json['departures']:
                flights_dict = {}
                
                # if flight['isCargo'] == False:
                
                # flights_dict['is cargo'] = []
                
                flights_dict['icao'] = icao
                
                # .get() is another way of ensuring our code doesn't break
                # in the previous 2 notebooks you learnt about 'if' (cities) and 'try/except' (weather)
                # .get() works similar, it will get the text if possible, if there is no text a None value will be inserted instead
                flights_dict['departure_time_local'] = flight['departure'].get('scheduledTimeLocal', None)
                flights_dict['departure_time_Utc'] = flight['departure'].get('scheduledTimeUtc', None)
                flights_dict['data_retrieved_on'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()
                
                # creating a list of dictionaries that all have the same keys
                # therefore the dataframe will include the same informations under the same category
                list_for_df.append(flights_dict)

    return pd.DataFrame(list_for_df)

## Creating the dataframe for pushing in two steps

In [ ]:
icaos = ['EDDB']
depart_df = flight_departures(icaos).head()

In [ ]:
depart_df

,icao,departure_time_local,departure_time_Utc,data_retrieved_on
0,EDDB,2022-11-30 18:25+01:00,2022-11-30 17:25Z,2022-11-30
1,EDDB,2022-11-30 18:40+01:00,2022-11-30 17:40Z,2022-11-30
2,EDDB,2022-11-30 17:45+01:00,2022-11-30 16:45Z,2022-11-30
3,EDDB,2022-11-30 18:45+01:00,2022-11-30 17:45Z,2022-11-30
4,EDDB,2022-11-30 19:45+01:00,2022-11-30 18:45Z,2022-11-30


### Adding the foreign key

In [ ]:
## inserting theempty column at the right place
airport_departures_df = depart_df.copy()
airport_departures_df.insert(1, "weather_FK", "", allow_duplicates=True)
airport_departures_df

## filling the column with values
for time in airport_departures_df.loc[:,"departure_time_local"]:
    w_t = datetime.strptime(time,"%Y-%m-%d %H:%M+01:00")
    # weather interval
    weath_int = (int(w_t.strftime("%H")) // 3)*3
    suffix = w_t.strftime("%Y-%m-%d ")
    # print(f"{suffix}{weath_int}:00:00")
    airport_departures_df["weather_FK"] = f"{suffix}{weath_int}:00:00"

### Cleaning all datetime values

In [ ]:
airport_departures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   icao                  5 non-null      object
 1   weather_FK            5 non-null      object
 2   departure_time_local  5 non-null      object
 3   departure_time_Utc    5 non-null      object
 4   data_retrieved_on     5 non-null      object
dtypes: object(5)
memory usage: 328.0+ bytes


In [ ]:
airport_departures_df["departure_time_local"] = pd.to_datetime(airport_departures_df["departure_time_local"])

In [ ]:
airport_departures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype                               
---  ------                --------------  -----                               
 0   icao                  5 non-null      object                              
 1   weather_FK            5 non-null      object                              
 2   departure_time_local  5 non-null      datetime64[ns, pytz.FixedOffset(60)]
 3   departure_time_Utc    5 non-null      object                              
 4   data_retrieved_on     5 non-null      object                              
dtypes: datetime64[ns, pytz.FixedOffset(60)](1), object(4)
memory usage: 328.0+ bytes


In [ ]:
airport_departures_df["departure_time_Utc"] = pd.to_datetime(airport_departures_df["departure_time_Utc"])

In [ ]:
airport_departures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype                               
---  ------                --------------  -----                               
 0   icao                  5 non-null      object                              
 1   weather_FK            5 non-null      object                              
 2   departure_time_local  5 non-null      datetime64[ns, pytz.FixedOffset(60)]
 3   departure_time_Utc    5 non-null      datetime64[ns, UTC]                 
 4   data_retrieved_on     5 non-null      object                              
dtypes: datetime64[ns, UTC](1), datetime64[ns, pytz.FixedOffset(60)](1), object(3)
memory usage: 328.0+ bytes


## double check results

In [ ]:
airport_departures_df

,icao,weather_FK,departure_time_local,departure_time_Utc,data_retrieved_on
0,EDDB,2022-11-30 18:00:00,2022-11-30 18:25:00+01:00,2022-11-30 17:25:00+00:00,2022-11-30
1,EDDB,2022-11-30 18:00:00,2022-11-30 18:40:00+01:00,2022-11-30 17:40:00+00:00,2022-11-30
2,EDDB,2022-11-30 18:00:00,2022-11-30 17:45:00+01:00,2022-11-30 16:45:00+00:00,2022-11-30
3,EDDB,2022-11-30 18:00:00,2022-11-30 18:45:00+01:00,2022-11-30 17:45:00+00:00,2022-11-30
4,EDDB,2022-11-30 18:00:00,2022-11-30 19:45:00+01:00,2022-11-30 18:45:00+00:00,2022-11-30


In [ ]:
airport_arrivals_df

,icao,weather_FK,arrival_time_local,arrival_time_Utc,data_retrieved_on
0,EDDB,2022-12-01 6:00:00,2022-12-01 07:55:00+01:00,2022-12-01 06:55:00+00:00,2022-11-30
1,EDDB,2022-12-01 6:00:00,2022-12-01 07:30:00+01:00,2022-12-01 06:30:00+00:00,2022-11-30
2,EDDB,2022-12-01 6:00:00,2022-12-01 07:50:00+01:00,2022-12-01 06:50:00+00:00,2022-11-30
3,EDDB,2022-12-01 6:00:00,2022-12-01 07:55:00+01:00,2022-12-01 06:55:00+00:00,2022-11-30
4,EDDB,2022-12-01 6:00:00,2022-12-01 07:55:00+01:00,2022-12-01 06:55:00+00:00,2022-11-30


# Pushing the data frames to mySQL

### Chosing the right instance by preparing the variables

In [ ]:
schema = "P3_Cities"   # name of the database you want to use here
host="city-project3-db.cf7wpiecfnwf.us-east-1.rds.amazonaws.com"        # to connect to your local server
user="admin"
password = PASSWORD # <----- your PASSWORD
port= 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

### Before the push 

### When pushing Dataframes to SQL they will become Tables in SQL.
###### When Pushing there is one requirement, the database/schema into the which data is going to be pushed has to be created in the respective instance/connection (to choose from mySQL "home"). There are three possible scenarios for pushing.
```
        P.1.  The Table hasn't been created in the DB: 
                then the push is gonna create it. 
        P.2.  The Table has been created without foreign keys:
                then the push is gonna add informaiton to the table.
        P.3.  The Table has been created with with foreign keys:
                  then the push can only happen, by respecting consitency rules of SQL.
```

#### both our dataframes have foreign keys, that is why case P3 applies here

##### we have to make sure, that all columns already exist in the mySQL database and that both, primary key and foreign key are already defined


In [ ]:
# Pushing the arrivals
airport_arrivals_df.to_sql('flight_arrivals',     # table name;
                       if_exists='append',      # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
                      con=con,               # con-> connection string;
                      index=False)           # index = False -> will not send index column to database

5

In [ ]:
# Pushing the departures
airport_departures_df.to_sql('flight_departures',     # table name;
                       if_exists='append',      # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
                      con=con,               # con-> connection string;
                      index=False)  

5